In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("ABHAY_API_KEY")

In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage


class AgentState(TypedDict):
    user_input: str
    agent_response: str
    interaction_count: int


llm = ChatGoogleGenerativeAI(
    model="gemini-3-flash-preview",
    temperature=0
)


def agent_node(state: AgentState) -> AgentState:
    response = llm.invoke([
        HumanMessage(content=state["user_input"])
    ])

    if isinstance(response.content, list):
        text = "".join(
            block["text"]
            for block in response.content
            if block.get("type") == "text"
        )
    else:
        text = response.content

    return {
        "user_input": state["user_input"],
        "agent_response": text,
        "interaction_count": state["interaction_count"] + 1
    }


graph = StateGraph(AgentState)
graph.add_node("agent", agent_node)
graph.set_entry_point("agent")
graph.add_edge("agent", END)
app = graph.compile()


interaction_count = 0

while True:
    user_input = input("Enter your question (type 'Exit' to quit): ")

    if user_input.lower() == "exit":
        break

    result = app.invoke({
        "user_input": user_input,
        "agent_response": "",
        "interaction_count": interaction_count
    })

    interaction_count = result["interaction_count"]
    print(result["agent_response"])
